In [1]:
# Run on different "full" depths
# Re-run roadmap experiments 
# Map all scRNA stuff

import os
import copy
import tempfile
import json
from subprocess import call
import wandb
from diConstants import (HG19_ALL_CHROMS, MM9_ALL_CHROMS,
    HG19_TRAIN_CHROMS, MM9_TRAIN_CHROMS,
    VALID_CHROMS, TEST_CHROMS) 

import models
import modelTemplates


C:\Users\Vadim\anaconda3\envs\coda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Using TensorFlow backend.


In [3]:
def run_model(model_params):
    m = models.SeqModel.instantiate_model(model_params)
    m.compile_and_train_model()
    results = m.evaluate_model()
    return results

# GM_MARKS = ['H3K27AC', 'H3K4ME1', 'H3K4ME3', 'H3K27ME3', 'H3K36ME3']
GM_MARKS = ['H3K27AC', 'H3K4ME1']


def test_GM18526():

    for test_cell_line in ['GM18526']:
        for subsample_target_string in ['0.5e6']:
            for predict_binary_output in [True, False]:    
                for output_mark in GM_MARKS:                            

                    model_params = modelTemplates.make_model_params(
                        model_library='keras',
                        model_class='SeqToPoint',
                        model_type='cnn',
                        model_specific_params={
                            'num_filters': 6,
                            'filter_length': 51
                        },
                        compile_params={            
                            'optimizer': 'adagrad'
                        },
                        dataset_params={
                            'train_dataset_name': 'GM12878_5+1marks-K4me3_all',
                            'test_dataset_name': '%s_5+1marks-K4me3_all' % test_cell_line, 
                            'num_train_examples': 1000,
                            'seq_length': 1001,
                            'peak_fraction': 0.5,                            
                            'train_X_subsample_target_string': subsample_target_string,
                            'num_bins_to_test': None,
                            'train_chroms': HG19_ALL_CHROMS,
                            'test_chroms': HG19_ALL_CHROMS,
                            'only_chr1': True
                        },
                        output_marks=[output_mark],
                        train_params={
                            'nb_epoch': 30,
                            'batch_size': 64
                        },
                        predict_binary_output=predict_binary_output,
                        zero_out_non_bins=True,
                        generate_bigWig=False)

                    group = "peaks" if predict_binary_output else "signal"
                    # Initilize a new wandb run
                    wandb.init(entity="vadim-farutin", project="coda",
                               config=model_params, reinit=True, group=group)
                    # wandb.watch_called = False # Re-run the model without restarting the runtime, unnecessary after our next release

                    results = run_model(model_params)

                    if predict_binary_output:
                        wandb.run.summary['train_samples_dn_AUC']              = results['train_samples']['dn']['AUC']
                        wandb.run.summary['train_samples_dn_AUPRC']            = results['train_samples']['dn']['AUPRC']
                        wandb.run.summary['train_samples_dn_Y_pos_frac']       = results['train_samples']['dn']['Y_pos_frac']
                        wandb.run.summary['train_samples_dn_precision_curves'] = results['train_samples']['dn']['precision_curves']
                        wandb.run.summary['train_samples_dn_recall_curves']    = results['train_samples']['dn']['recall_curves']

                        wandb.run.summary['test_samples_dn_AUC']              = results['test_results'][0]['samples']['dn']['AUC']
                        wandb.run.summary['test_samples_dn_AUPRC']            = results['test_results'][0]['samples']['dn']['AUPRC']
                        wandb.run.summary['test_samples_dn_Y_pos_frac']       = results['test_results'][0]['samples']['dn']['Y_pos_frac']
                        wandb.run.summary['test_samples_dn_precision_curves'] = results['test_results'][0]['samples']['dn']['precision_curves']
                        wandb.run.summary['test_samples_dn_recall_curves']    = results['test_results'][0]['samples']['dn']['recall_curves']

                        wandb.run.summary['test_genome_dn_AUC']              = results['test_results'][0]['genome']['dn_all']['chr1']['AUC']
                        wandb.run.summary['test_genome_dn_AUPRC']            = results['test_results'][0]['genome']['dn_all']['chr1']['AUPRC']
                        wandb.run.summary['test_genome_dn_Y_pos_frac']       = results['test_results'][0]['genome']['dn_all']['chr1']['Y_pos_frac']
                        wandb.run.summary['test_genome_dn_precision_curves'] = results['test_results'][0]['genome']['dn_all']['chr1']['precision_curves']
                        wandb.run.summary['test_genome_dn_recall_curves']    = results['test_results'][0]['genome']['dn_all']['chr1']['recall_curves']

                    else:
                        wandb.run.summary['train_samples_dn_MSE']      = results['train_samples']['dn']['MSE']
                        wandb.run.summary['train_samples_dn_true_var'] = results['train_samples']['dn']['true_var']
                        wandb.run.summary['train_samples_dn_pearsonR'] = results['train_samples']['dn']['pearsonR']

                        wandb.run.summary['test_samples_dn_MSE']      = results['test_results'][0]['samples']['dn']['MSE']
                        wandb.run.summary['test_samples_dn_true_var'] = results['test_results'][0]['samples']['dn']['true_var']
                        wandb.run.summary['test_samples_dn_pearsonR'] = results['test_results'][0]['samples']['dn']['pearsonR']

                        wandb.run.summary['test_genome_dn_all_MSE']      = results['test_results'][0]['genome']['dn_all']['chr1']['MSE']
                        wandb.run.summary['test_genome_dn_all_true_var'] = results['test_results'][0]['genome']['dn_all']['chr1']['true_var']
                        wandb.run.summary['test_genome_dn_all_pearsonR'] = results['test_results'][0]['genome']['dn_all']['chr1']['pearsonR']

                        wandb.run.summary['test_genome_dn_peaks_MSE']      = results['test_results'][0]['genome']['dn_peaks']['chr1']['MSE']
                        wandb.run.summary['test_genome_dn_peaks_true_var'] = results['test_results'][0]['genome']['dn_peaks']['chr1']['true_var']
                        wandb.run.summary['test_genome_dn_peaks_pearsonR'] = results['test_results'][0]['genome']['dn_peaks']['chr1']['pearsonR']

                    wandb.join()
                    # wandb.uninit()


In [4]:
test_GM18526()

Logging results to <a href="https://wandb.com" target="_blank">Weights & Biases</a> <a href="https://docs.wandb.com/integrations/jupyter.html" target="_blank">(Documentation)</a>.<br/>
                Project page: <a href="https://app.wandb.ai/vadim-farutin/coda" target="_blank">https://app.wandb.ai/vadim-farutin/coda</a><br/>
                Run page: <a href="https://app.wandb.ai/vadim-farutin/coda/runs/lnvwwh14" target="_blank">https://app.wandb.ai/vadim-farutin/coda/runs/lnvwwh14</a><br/>

Initialized model with parameters:
{
    "model_library": "keras",
    "model_class": "SeqToPoint",
    "model_type": "cnn",
    "scale_input": "01",
    "random_seed": 0,
    "generate_bigWig": false,
    "predict_binary_output": true,
    "zero_out_non_bins": true,
    "dataset_params": {
        "train_dataset": {
            "dataset_name": "GM12878_5+1marks-K4me3_all",
            "num_examples": 1000,
            "X_subsample_target_string": "0.5e6",
            "Y_subsample_target_string": null,
            "random_seed": 0,
            "normalization": "arcsinh",
            "peak_fraction": 0.5,
            "chroms": [
                "chr1",
                "chr2",
                "chr3",
                "chr4",
                "chr5",
                "chr6",
                "chr7",
                "chr8",
                "chr9",
                "chr10",
                "chr11",
                "chr12",
                "chr13",
                "chr14",
                "chr15"

C:\Users\Vadim\Documents\PycharmProjects\coda\models.py:868: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(6, 51, input_shape=(None, 6), padding="same", kernel_initializer="uniform")`
  border_mode='same'))
C:\Users\Vadim\Documents\PycharmProjects\coda\models.py:880: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(1, 1001, padding="valid", kernel_initializer="uniform")`
  border_mode='valid'))


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


C:\Users\Vadim\Documents\PycharmProjects\coda\models.py:333: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  **self.model_params['train_params'])


...

MSE 0.774 (true var 0.564), pearsonR 0.492
Test chr1, 9.97E+06 bins - Original, only peaks:


MSE 3.720 (true var 0.647), pearsonR 0.443


Test chr1, 9.97E+06 bins - Denoised, all signal:


MSE 0.308 (true var 0.564), pearsonR 0.734
Test chr1, 9.97E+06 bins - Denoised, only peaks:


MSE 0.754 (true var 0.647), pearsonR 0.611
final results {'orig_all': {'chr1': {'MSE': [0.7744798305872056], 'true_var': [0.5642468483899405], 'pearsonR': [0.4922631954177217]}}, 'dn_all': {'chr1': {'MSE': [0.3084584836576798], 'true_var': [0.5642468483899405], 'pearsonR': [0.7335485703998634]}}, 'orig_peaks': {'chr1': {'MSE': [3.72038587307587], 'true_var': [0.6465770880301691], 'pearsonR': [0.4426086546309038]}}, 'dn_peaks': {'chr1': {'MSE': [0.7535669620872818], 'true_var': [0.6465770880301691], 'pearsonR': [0.6107308282615863]}}}
